# EPhys Data Analysis Tutorial

This tutorial demonstrates a step-by-step approach to processing electrophysiology (EPhys) data for analysis using Meghan's multirecording_spikeanalysis script, the RCE Pilot 2 behavior spreadsheet, and directories of phy folders (with spike_times.npy, spike_clusters.npy, & cluster_group.tsv) for each ephys recording.

## Setup

Import all the libraries you'll be using, including Meghan's multirecording_spikeanalysis.py:

In [1]:
import pandas as pd
import numpy as np
import ast
import pickle
from pathlib import Path
import multirecording_spikeanalysis as spike
from statistics import mean, StatisticsError

## Data Loading
First, we load the relevant EPhys data from the RCE Pilot 2 spreadsheet:

In [2]:
cols = ['condition ', 'session_dir', 'all_subjects', 'tone_start_timestamp', 'tone_stop_timestamp']

# Load the data
df = pd.read_excel('rce_pilot_2_per_video_trial_labels.xlsx', usecols=cols, engine='openpyxl')

## Preprocessing

Next, we rearrange the spreadsheet in order for it to prepare it for ephys recordings:

In [3]:
df2 = df.dropna() # Drop the rows missing data
df3 = df2.copy()
df3['all_subjects'] = df3['all_subjects'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x) # Make the 'all_subjects' column readable as a list
df4 = df3[df3['all_subjects'].apply(lambda x: len(x) < 3)] # Ignore novel sessions for now

## Data Structuring
We'll structure the data into a new DataFrame that aligns with our analysis goals:

In [4]:
# Initialize an empty list to collect data for the new DataFrame
new_df_data = []

for _, row in df4.iterrows():
    session_dir = row['session_dir']
    subjects = row['all_subjects']
    condition = row['condition ']

    # Split session_dir on '_subj_' and take the first part only
    # This ensures everything after '_subj_' is ignored
    base_session_dir = session_dir.split('_subj_')[0]

    for subject in subjects:
        subject_formatted = subject.replace('.', '-')
        # Append formatted subject to the base session_dir correctly
        subj_recording = f"{base_session_dir}_subj_{subject_formatted}"
        new_df_data.append({
            'session_dir': session_dir,
            'subject': subject,
            'subj_recording': subj_recording,
            'condition': condition if condition in ['rewarded', 'omission', 'both_rewarded', 'tie'] else ('win' if str(condition) == str(subject) else 'lose'),
            'tone_start_timestamp': row['tone_start_timestamp'],
            'tone_stop_timestamp': row['tone_stop_timestamp']
        })

# Convert list to DataFrame
new_df = pd.DataFrame(new_df_data)
new_df = new_df.drop_duplicates()

## Timestamp Dictionary Preparation
Prepare dictionaries of event timestamps to match with the ephys recordings:

In [5]:
# Prepare timestamp_dicts from new_df
timestamp_dicts = {}
for _, row in new_df.iterrows():
    key = row['subj_recording']
    condition = row['condition']
    timestamp_start = int(row['tone_start_timestamp']) // 20
    timestamp_end = int(row['tone_stop_timestamp']) // 20
    tuple_val = (timestamp_start, timestamp_end)

    if key not in timestamp_dicts:
        timestamp_dicts[key] = {cond: [] for cond in ['rewarded', 'win', 'lose', 'omission', 'both_rewarded', 'tie']}
    timestamp_dicts[key][condition].append(tuple_val)

# Convert lists in timestamp_dicts to numpy arrays
for subj_recording in timestamp_dicts:
    for condition in timestamp_dicts[subj_recording]:
        timestamp_dicts[subj_recording][condition] = np.array(timestamp_dicts[subj_recording][condition], dtype=np.int64)

## EPhys Recording Collection
Load EPhys recordings:

In [6]:
# Construct the path in a platform-independent way (HiPerGator or Windows)
ephys_path = Path('.') / 'export' / 'updated_phys' / 'non-novel' / 'all_non_novel'

ephys_data = spike.EphysRecordingCollection(str(ephys_path))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230618_100636_standard_comp_to_omission_D2_subj_1-1_t1b2L_box2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230618_100636_standard_comp_to_omission_D2_subj_1-4_t4b3L_box1_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230620_114347_standard_comp_to_omission_D4_subj_1-1_t1b2L_box_2_merged.rec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec
<class

## Assign Dictionaries to Collection
Create dictionaries for each recording, and assign it and the subject number to the recording:

In [7]:
for recording in ephys_data.collection.keys():
    # Check if the recording key (without everything after subject #) is in timestamp_dicts
    start_pos = recording.find('subj_')
    # Add the length of 'subj_' and 3 additional characters to include after 'subj_'
    end_pos = start_pos + len('subj_') + 3
    # Slice the recording key to get everything up to and including the subject identifier plus three characters
    recording_key_without_suffix = recording[:end_pos]
    if recording_key_without_suffix in timestamp_dicts:
        # Assign the corresponding timestamp_dicts dictionary to event_dict
        ephys_data.collection[recording].event_dict = timestamp_dicts[recording_key_without_suffix]
        
        # Extract the subject from the recording key
        start = recording.find('subj_') + 5  # Start index after 'subj_'
        subject = recording[start:start+3]
        
        # Assign the extracted subject
        ephys_data.collection[recording].subject = subject

## Analysis Initialization
Finally, initialize the spike analysis with the organized EPhys data (it would be nice to pickle this, but even with 4 CPUs & 64 GB RAM, it still crashed trying to pickle):

In [8]:
spike_analysis = spike.SpikeAnalysis_MultiRecording(ephys_data, timebin = 5, smoothing_window=250, ignore_freq = 0.5)

All set to analyze


### The steps before this are done on almost every notebook, and it would be nice to pickle `ephys_data` or `spike_analysis` (spike_analysis is basically just ephys_data with even more data initialized), but it is too big to pickle

### Now I'll show you what the class, class objects, class methods, and original data look like

In [9]:
# This will create a variable that will make selecting a single recording easier. It's simply pointing to the part of the class where the recording names are stored
recordings = spike_analysis.ephyscollection.collection

In [10]:
# Creating a variable of 1 recording
recording_name = '20230618_100636_standard_comp_to_omission_D2_subj_1-4_t4b3L_box1_merged.rec'
recording1 = recordings.get(recording_name)

if recording1 is None:
    print(f"Recording named {recording_name} not found.")
else:
    print(f"Recording {recording_name} successfully retrieved.")

Recording 20230618_100636_standard_comp_to_omission_D2_subj_1-4_t4b3L_box1_merged.rec successfully retrieved.


In [11]:
if recording1:
    print(dir(recording1))  # Lists all methods and attributes of the recording

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'event_dict', 'freq_dict', 'get_spike_specs', 'get_unit_labels', 'get_unit_timestamps', 'labels_dict', 'path', 'sampling_rate', 'spiketrain', 'subject', 'timestamps_var', 'unit_array', 'unit_firing_rate_array', 'unit_firing_rates', 'unit_spiketrains', 'unit_timestamps', 'wilcox_dfs', 'zscored_events']


In [12]:
# I've already created and assigned 'event dictionaries' to each recording. 
# This is what the 'win' event_dict actually looks like. It's timestamps for the beginning and end of each time a win occurred.
recording1.event_dict['win']

array([[  54962,   64962],
       [ 379962,  389962],
       [ 484962,  494962],
       [ 579962,  589962],
       [ 654962,  664962],
       [1554961, 1564961]], dtype=int64)

In [13]:
preevent = recording1.event_dict['win'] - 10000

In [14]:
# Here I manually created a pre-event dictionary, this is essentially what class methods do when asking for a pre-event window
preevent

array([[  44962,   54962],
       [ 369962,  379962],
       [ 474962,  484962],
       [ 569962,  579962],
       [ 644962,  654962],
       [1544961, 1554961]], dtype=int64)

### The class automatically stores smoothed firing rate bins for each unit for each recording
### I created the class with:
spike_analysis = spike.SpikeAnalysis_MultiRecording(ephys_data, timebin = 5, smoothing_window=250, ignore_freq = 0.5)
### So each timebin is 5ms, and firing rate for each timebin is the average firing rate per second of the surrounding 250ms.
### Example, imagine the array below is the spiketrain, or the actual count of spikes per window, and we'll use 100ms timebins:
[1, 5, 2, 1, 4, 5, 6, 8, 7, 4]
### If we did a 500ms smoothing window, you'd look at each bin and average it with the 2 to the left and 2 to the right.
### Ignore the first 2 and last 2 bins for now:
timebin 3: 1+5+2+1+4 = 13 spikes during a 500 ms window, so to convert to firing rate/second -> 13*2 = 26\
timebin 4: 5+2+1+4+5 = 17 * 2 = 34\
timebin 5: 2+1+4+5+6 = 18 * 2 = 36\
timebin 6: 1+4+5+6+8 = 24 * 2 = 48\
timebin 7: 4+5+6+8+7 = 30 * 2 = 60\
timebin 8: 5+6+8+7+4 = 30 * 2 = 60
### The function within the class that creates the firing rate, makes sure to have the output be the same length as the input, so it handles the beginning and the end by just weighting in a non-uniform way and just uses the windows it can. So the example output firing rate would be:
[16, 18, 26, 34, 36, 48, 60, 60, 50, 38]

In [15]:
recording1.unit_firing_rates

{65: array([8., 8., 8., ..., 0., 0., 0.]),
 123: array([8., 8., 8., ..., 0., 0., 0.]),
 103: array([8., 8., 8., ..., 4., 4., 4.]),
 83: array([4., 4., 4., ..., 8., 8., 8.]),
 118: array([0., 0., 0., ..., 0., 0., 0.]),
 93: array([0., 0., 0., ..., 4., 4., 4.]),
 99: array([0., 0., 0., ..., 4., 4., 4.]),
 105: array([0., 0., 0., ..., 0., 0., 0.]),
 87: array([0., 0., 0., ..., 4., 4., 4.]),
 19: array([0., 0., 0., ..., 0., 0., 0.]),
 9: array([0., 0., 0., ..., 0., 0., 0.])}

### Unit Firing Rates

The `unit_firing_rates` dictionary contains arrays of firing rates for each unit, smoothed and computed over the specified time bins. Each unit's associated array represents the firing rate over the entire recording session. The units are normalized over a `5 ms` time bin with a `250 ms` smoothing window to handle fluctuations in firing rate.

In [16]:
len(recording1.unit_firing_rates[65])

670988

In [17]:
len(recording1.unit_firing_rates[123])

670988

### Analysis of Firing Rate Duration

The length of each unit's firing rate array tells you the total number of time bins for that unit, but they should be the same for every unit. For example, the length `670,988` for unit `65` indicates the total number of `5 ms` bins, which matches unit `123` as well.\
\
If we wanted to, we could calculate the length of the whole recording with this information. `5 ms * 670,988 bins = 3,354,940 ms -> 3,354 s -> 56 min`

In [18]:
spike_analysis = spike.SpikeAnalysis_MultiRecording(ephys_data, timebin = 100, smoothing_window=250, ignore_freq = 0.5)

All set to analyze


### Re-initializing Spike Analysis

Here, we re-initialize the `SpikeAnalysis_MultiRecording` with a different size `timebin`. Now we'll see how this affects results.

In [19]:
recordings = spike_analysis.ephyscollection.collection

recording_name = '20230618_100636_standard_comp_to_omission_D2_subj_1-4_t4b3L_box1_merged.rec'
recording1 = recordings.get(recording_name)

In [20]:
recording1.unit_spiketrains

{65: array([2, 2, 0, ..., 0, 0, 0], dtype=int64),
 123: array([2, 0, 0, ..., 0, 0, 0], dtype=int64),
 103: array([2, 2, 1, ..., 1, 0, 1], dtype=int64),
 83: array([0, 1, 1, ..., 1, 3, 1], dtype=int64),
 118: array([0, 1, 0, ..., 0, 0, 0], dtype=int64),
 93: array([0, 0, 0, ..., 1, 0, 1], dtype=int64),
 99: array([0, 0, 0, ..., 1, 1, 1], dtype=int64),
 105: array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 87: array([0, 0, 0, ..., 0, 1, 0], dtype=int64),
 19: array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 9: array([0, 0, 0, ..., 0, 0, 0], dtype=int64)}

In [21]:
len(recording1.unit_spiketrains[65])

33549

In [22]:
recording1.unit_firing_rates

{65: array([10., 20., 10., ...,  0.,  0.,  0.]),
 123: array([10., 10.,  0., ...,  0.,  0.,  0.]),
 103: array([10., 20., 15., ..., 10.,  5.,  5.]),
 83: array([ 0.,  5., 10., ..., 15., 20., 20.]),
 118: array([0., 5., 5., ..., 0., 0., 0.]),
 93: array([0., 0., 0., ..., 5., 5., 5.]),
 99: array([ 0.,  0.,  0., ...,  5., 10., 10.]),
 105: array([0., 0., 0., ..., 0., 0., 0.]),
 87: array([0., 0., 0., ..., 0., 5., 5.]),
 19: array([0., 0., 0., ..., 0., 0., 0.]),
 9: array([0., 0., 0., ..., 0., 0., 0.])}

In [23]:
len(recording1.unit_firing_rates[65])

33549

### Spike Train Data

The `spiketrain` array represents the total number of spikes recorded in each time bin across all units, while `unit_spiketrains` provides a breakdown of spikes per unit for the same bins. Analyzing these arrays helps in understanding the temporal patterns of activity and the synchrony between different units during the experiment.

### Adjusted Unit Firing Rates

After adjusting `timebin` we saw that the length of timebins reduced dramatically going from `5 ms` timebins to `100 ms` time bins, but we should be able to re-calculate the length of the whole recording to be the same as in the previous calculation: `100 ms * 33,549 = 3,354,900 ms -> 3,354 s -> 56 min`\
\
Also, note the difference between `unit_firing_rates` and `unit_spiketrains`. `unit_spiketrains` is the raw count of spikes per timebin, `unit_firing_rates` is smoothed with surrounding bins and is in the format of `spikes/second` as opposed to `spikes/timebin`.

In [24]:
np.mean(recording1.unit_spiketrains[65])

0.2686816298548392

In [25]:
np.mean(recording1.unit_firing_rates[65])

2.686816298548392

### Mean Spike Rate and Firing Rate Calculation

Note that the mean `unit_firing_rates` for each unit is `unit_spiketrains` x 10 since each timebin is 100ms, thus 10 timebins/second. 

In [26]:
def get_firing_rate(spiketrain, smoothing_window, timebin):
    """
    calculates firing rate (spikes/second)

    Args (3 total, 1 required):
        spiketrain: numpy array, in timebin (ms) bins
        smoothing_window: int, default=250, smoothing average window (ms)
            min smoothing_window = 1
        timebin: int, default = 1, timebin (ms) of spiketrain

    Return (1):
        firing_rate: numpy array of firing rates in timebin sized windows

    """
    smoothing_bins = int(smoothing_window / timebin)
    weights = np.ones(smoothing_bins) / smoothing_bins * 1000 / timebin
    firing_rate = np.convolve(spiketrain, weights, mode="same")

    return firing_rate

fr_65_man = get_firing_rate(recording1.unit_spiketrains[65], 250, 100)

### Custom Firing Rate Calculation Function

This function `get_firing_rate`, is pulled directly from `multirecording_spikeanalysis.py`.

In [27]:
fr_65_man

array([10., 20., 10., ...,  0.,  0.,  0.])

In [28]:
len(fr_65_man)

33549

### Length of Manually Computed Firing Rate Array

The length of the manually computed firing rate array for unit 65 indicates the total number of timebins analyzed after adjusting the analysis parameters. This value should be consistent with the length of `unit_spiketrains` to ensure that all spikes have been accounted for in the firing rate calculation.

In [29]:
np.mean(fr_65_man)

2.686816298548392

In [39]:
smoothing_window = 300
timebin=100

smoothing_bins = int(smoothing_window / timebin)
weights = np.ones(smoothing_bins) / smoothing_bins * 1000 / timebin

In [40]:
weights

array([3.33333333, 3.33333333, 3.33333333])

### Weight Array for Smoothing Calculation

The `weights` array represents the normalization factors used in the convolution operation to smooth the spiketrain data. This array helps you understand how the averaging window influences the resultant firing rate, ensuring that each spike's contribution is appropriately scaled over the smoothing window.

In [41]:
recording1.event_dict

{'rewarded': array([[1854961, 1864961],
        [1914961, 1924961],
        [1969961, 1979961],
        [2034961, 2044961],
        [2089961, 2099961],
        [2139961, 2149961],
        [2189961, 2199961],
        [2414961, 2424961],
        [2534961, 2544961],
        [2644961, 2654961],
        [2729961, 2739961],
        [2849961, 2859961],
        [2974961, 2984961],
        [3034961, 3044961],
        [3109961, 3119961]], dtype=int64),
 'win': array([[  54962,   64962],
        [ 379962,  389962],
        [ 484962,  494962],
        [ 579962,  589962],
        [ 654962,  664962],
        [1554961, 1564961]], dtype=int64),
 'lose': array([[ 174962,  184962],
        [ 289962,  299962],
        [ 434962,  444962],
        [ 759962,  769962],
        [ 809962,  819962],
        [ 889962,  899962],
        [ 954962,  964962],
        [1019962, 1029962],
        [1069962, 1079962],
        [1139962, 1149962],
        [1234962, 1244962],
        [1314962, 1324962],
        [1384962, 1

### Event Dictionary Overview

The `event_dict` stores arrays of start and stop timestamps for different behavioral events identified during the recording. These include `rewarded`, `win`, `lose`, and empty arrays that aren't applicable for this recording.

In [42]:
import math

events = recording1.event_dict['win']

event_snippets = []
pre_window = math.ceil(10 * 1000)
post_window = math.ceil(0 * 1000)
equalize = 10 * 1000
e_length = equalize + post_window + pre_window

### Extracting Event-Related Neural Activity Snippets

The `event_snippets` code block, representative of a function or class method from the `multirecording_spikeanalysis` script, is designed to extract the window of timebins specific to what you're interested in, ex: an event.

Here's how it works:
- **Event Identification**: First, it retrieves the `win` event timestamps from `event_dict`.
- **Window Setup**: It sets up windows around these events. In this example, the pre-event window is 10 seconds, and there is no post-event window. This setup can be adjusted depending on the analysis needs.
- **Snippet Extraction**: For each unit, the method iterates through the event timestamps, using the pre-defined windows to slice the corresponding segments from the firing rate array. 
- **Length Check**: It also ensures that each snippet has the correct length, matching the expected duration (sum of pre-event, event, and post-event durations), thereby standardizing the snippet lengths for consistent analysis.

In [44]:
events = recording1.event_dict['win']

event_snippets = []
pre_window = math.ceil(10 * 1000)  # 10 seconds pre-event window
post_window = math.ceil(0 * 1000)  # 0 seconds post-event window, adjust if needed
equalize = 10 * 1000  # 10 seconds equalize duration
e_length = equalize + post_window + pre_window  # Total length of the snippet
timebin = 1000  # Assuming timebin is 1000 ms; adjust as per your setup

for unit_key in recording1.unit_firing_rates.keys():
    unit_rates = recording1.unit_firing_rates[unit_key]  # Retrieve the firing rates array
    for i in range(events.shape[0]):
        pre_event = math.ceil((events[i][0] - pre_window) / timebin)
        post_event = math.ceil((events[i][0] + post_window + equalize) / timebin)
        if unit_rates.ndim == 1:  # Check if it's a 1D array
            event_snippet = unit_rates[pre_event:post_event]
            if len(event_snippet) == e_length / timebin:
                event_snippets.append(event_snippet)
        else:  # Handling multidimensional arrays
            event_snippet = unit_rates[:, pre_event:post_event]
            if event_snippet.shape[1] == e_length / timebin:
                event_snippets.append(event_snippet)


In [51]:
event_snippets[0]

array([10., 10., 10., 10., 10.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,
       10.,  5.,  0., 15., 20., 10., 15.])

In [46]:
len(event_snippets)

66

In [47]:
len(event_snippets[0])

20

### Statistical Analysis of Event-Related Neural Activity

Here, we perform a Wilcoxon signed-rank test to compare the firing rates during specific events against a baseline. The dataframe `df` summarizes the test statistics and p-values for each unit, indicating whether changes in neural activity are statistically significant. This method helps identify units with significant alterations in activity that correlate with behavioral events.

In [49]:
df = spike_analysis.__wilcox_baseline_v_event_stats__(recording_name, recording1, event='win', equalize=10, baseline_window=10, offset=0, exclude_offset=False, save=False)

In [50]:
df

,Wilcoxon Stat,p value,event1 vs event2
65,6.0,0.43750,not significant
123,6.0,0.43750,not significant
103,3.0,0.15625,not significant
83,0.0,0.03125,decreases
118,0.0,0.03125,decreases
93,0.0,0.03125,decreases
99,5.0,0.31250,not significant
105,5.0,0.62500,not significant
87,5.0,0.31250,not significant
19,0.0,0.06250,not significant


### Wilcoxon Limitations
Wilcoxon can technically be performed on samples of n < 6, but they're incapable of having a signficant p-value. This subject had 6 'win' events during this recording, but let's look at what would happen if it only had 5.

In [55]:
recording1.event_dict['win']

array([[  54962,   64962],
       [ 379962,  389962],
       [ 484962,  494962],
       [ 579962,  589962],
       [ 654962,  664962],
       [1554961, 1564961]], dtype=int64)

In [56]:
recording1.event_dict['win'] = recording1.event_dict['win'][:-1]

In [57]:
recording1.event_dict['win']

array([[ 54962,  64962],
       [379962, 389962],
       [484962, 494962],
       [579962, 589962],
       [654962, 664962]], dtype=int64)

In [58]:
df = spike_analysis.__wilcox_baseline_v_event_stats__(recording_name, recording1, event='win', equalize=10, baseline_window=10, offset=0, exclude_offset=False, save=False)

Wilcoxon can't be done on 20230618_100636_standard_comp_to_omission_D2_subj_1-4_t4b3L_box1_merged.rec win, because <6 samples


### Wilcoxon Small Sample Examples

In [61]:
from scipy.stats import wilcoxon

arraya = [1,2,3,4,5,6]
arrayb = [7,8,9,9,9,9]
test_stat, test_p_value = wilcoxon(arraya, arrayb)
test_p_value

0.03125

In [62]:
arraya = [1,2,3,4,5]
arrayb = [7,8,9,9,9]
test_stat, test_p_value = wilcoxon(arraya, arrayb)
test_p_value

0.0625

In [63]:
arraya = [1,1,1,1,1]
arrayb = [9,9,9,9,9]
test_stat, test_p_value = wilcoxon(arraya, arrayb)
test_p_value

0.0625

In [64]:
arraya = [1,1,9,1,1]
arrayb = [9,9,1,9,9]
test_stat, test_p_value = wilcoxon(arraya, arrayb)
test_p_value

0.3125

As you can see, it doesn't matter what the actual difference is between 2 arrays is, for 5 samples, the smallest possible p-value is 0.0625